In [1]:
# import statements
from time import sleep
import json
from json import dumps
from kafka import KafkaProducer
import random
from datetime import datetime
import datetime as dt
import csv
import pandas as pd
import pprint


In [2]:
# climate_streaming = []
# with open("FIT3182_Assignment/climate_streaming.csv", 'r') as file:
#     csvreader = csv.reader(file)
#     header_climate_streaming = next(csvreader)
#     for row in csvreader:
#         climate_streaming.append(row)
        
# climate_streaming = pd.DataFrame(climate_streaming)
# climate_streaming.columns = header_climate_streaming

climate_streaming = pd.read_csv("FIT3182_Assignment/climate_streaming.csv")
climate_streaming.head()

,latitude,longitude,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation,GHI_w/m2
0,-37.623,149.323,19,56.8,7.9,11.1,0.00I,154
1,-38.038,142.986,15,50.7,9.2,13.0,0.02G,128
2,-37.950,142.366,16,53.6,8.1,15.0,0.00G,133
3,-38.231,147.172,24,61.6,7.7,14.0,0.00I,186
4,-37.903,145.250,24,62.3,7.0,13.0,0.00I,185


In [3]:
import pymongo
from pymongo import MongoClient

client = MongoClient ()
db = client.fit3182_assignment_db
merged_hist = db.merged_hist

date = (list(merged_hist.aggregate([
        { '$unwind' : '$Hotspot' },
        { '$sort' : { 'Hotspot.datetime' : -1}},
        { '$limit' : 1},
        { '$project' : {'Hotspot.datetime' : 1 , '_id' : 0}}
         ])))

temp = date[0]

l_date = temp['Hotspot']

latest_date = l_date['datetime']
latest_date = latest_date.replace("T"," ")
latest_date = latest_date.replace("-","/")
latest_date = datetime.strptime(latest_date,"%Y/%m/%d %H:%M:%S")
latest_date

datetime.datetime(2021, 12, 27, 4, 16, 51)

In [4]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        #str_value = json.dumps(value)
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        

In [5]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [6]:
if __name__ == '__main__':
   
    topic = 'Assignment'
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    row_no = climate_streaming.shape[0]
    latest_date = latest_date + dt.timedelta(days = 1)
    
    
    for e in range(20):
        
        latest_date = str(latest_date.strftime("%d/%m/%Y, %H:%M:%S"))
        row = random.randrange(0,(row_no - 1))
        climate_streaming.loc[row,"datetime"] = latest_date
        climate_streaming.loc[row,"key"] = 'Producer01'
        data = climate_streaming.loc[row,]
        data = data.to_json()
        publish_message(producer, topic, 'Producer01', data)
        latest_date = datetime.strptime(latest_date,"%d/%m/%Y, %H:%M:%S") + dt.timedelta(seconds = 10)
        sleep(1)

Publishing records..
Message published successfully. Data: {"latitude":-37.954,"longitude":143.918,"air_temperature_celcius":20,"relative_humidity":59.7,"windspeed_knots":8.6,"max_wind_speed":15.9,"precipitation ":" 0.01G","GHI_w\/m2":158,"datetime":"28\/12\/2021, 04:16:51","key":"Producer01"}
Message published successfully. Data: {"latitude":-37.6745,"longitude":142.9848,"air_temperature_celcius":15,"relative_humidity":44.5,"windspeed_knots":10.2,"max_wind_speed":14.0,"precipitation ":" 0.00G","GHI_w\/m2":134,"datetime":"28\/12\/2021, 04:17:01","key":"Producer01"}
Message published successfully. Data: {"latitude":-38.527,"longitude":143.493,"air_temperature_celcius":19,"relative_humidity":51.5,"windspeed_knots":10.2,"max_wind_speed":20.0,"precipitation ":" 0.00I","GHI_w\/m2":161,"datetime":"28\/12\/2021, 04:17:11","key":"Producer01"}
Message published successfully. Data: {"latitude":-35.953,"longitude":141.078,"air_temperature_celcius":12,"relative_humidity":47.2,"windspeed_knots":8.8